In [1]:
from Py6S import *
import numpy as np
%matplotlib widget
import ipywidgets as widgets
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
import scipy

In [2]:
def wattsToPhotons(radwatts, waves):
    return radwatts / (1.98644582e-25 / (waves * 1e-6))


def convertRadAtSatLevelToPhotonsCapturedBySensor(
    radwatts,
    waves,
    apeture,
    efl,
    pixelsize,
    int_time,
    dlam,
    QE,
    inst_trans,
    anamorphic_factor=1,
):
    phots = wattsToPhotons(radwatts, waves)

    fnum = efl / apeture

    if type(pixelsize) is tuple:
        A = pixelsize[0] * pixelsize[1]
    else:
        A = pixelsize ** 2

    # SLDangle=apeture**2 * np.pi / (4*efl**2)

    SLDangle = np.pi / (2 * fnum) ** 2 / anamorphic_factor

    captured_phots = phots * int_time * A * SLDangle * dlam * inst_trans * QE

    return captured_phots


CMV2000_QE_RAW = np.genfromtxt("detector_qe/cmv2000qe.csv", delimiter=",")
IMX174_QE_RAW = np.genfromtxt("detector_qe/imx174qe.csv", delimiter=",")
IMX273_QE_RAW = np.genfromtxt("detector_qe/imx273qe.csv", delimiter=",")

CMV2000_QE = interp1d(CMV2000_QE_RAW[:, 0] / 1000, CMV2000_QE_RAW[:, 1] / 100)
IMX174_QE = interp1d(IMX174_QE_RAW[:, 0] / 1000, IMX174_QE_RAW[:, 1] / 100)
IMX273_QE = interp1d(IMX273_QE_RAW[:, 0] / 1000, IMX273_QE_RAW[:, 1] / 100)

QE = {"CMV2000": CMV2000_QE, "IMX174": IMX174_QE, "IMX273": IMX273_QE}


# Run 6SV simulation to estimate radiance received by sensor.
Uses the Py6SV wrapper and mostly default values for now. Geometry is set to obsevere Jervis Bay at 11am, nadir pointing (I think).

In [19]:
try:
    data = np.load("groundsixsmodel.npz")
    wv = data["wv"]
    res = data["res"]
except:
    s = SixS()

    # s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical)

    s.altitudes.set_sensor_sea_level()
    s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.MidlatitudeSummer)
    # s.ground_reflectance = GroundReflectance.HomogeneousLambertian(GroundReflectance.GreenVegetation)
    s.geometry = Geometry.User()
    s.geometry.from_time_and_location(
        -35.0481, 150.7447, "2021-01-01T00:00:00.0", 0, 0
    )  #% 11am AEDT above jervis bay.

    wv, res = SixSHelpers.Wavelengths.run_wavelengths(
        s, np.arange(0.430, 0.915, 0.005), output_name="pixel_radiance"
    )

    np.savez("groundsixsmodel.npz", wv=wv, res=res)



Running for many wavelengths - this may take a long time


# Interactive S/N calc

The papramets can be adjusted in the plot ot the effect on the S/N. 
- *Aperture* is the diameter of the entrance aperture of the sensor in mm
- *EFL* is effiective focal length in mm
- *pixelsize* is the size of a spatial pixel on the sensor in micrometers. (i.e. raw pixel size on the ximea linescane sensor or the slit size in spectrograh based design (note I assume in that case Entenue is conserved by the spec).
- *int_time* is the inegration time of the sensor in ms.
- *dlam* is the spectral resolution of the instrument (or the width of spectral bands).

In [44]:
plt.close('all') 
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))

ax.grid(True)
ax.autoscale(True, "both", True)

apeture = widgets.FloatSlider(value=4, min=1, max=200.0, step=1, description="Apeture:")
efl = widgets.FloatSlider(
    value=16, min=1, max=3000.0, step=1, description="Focal Lengh:"
)
pixel_size_x = widgets.FloatSlider(
    value=25, min=1, max=60, step=0.1, description="Pixel Size X:"
)
pixel_size_y = widgets.FloatSlider(
    value=3.45, min=1, max=60, step=0.1, description="Pixel Size Y:"
)
int_time_widget = widgets.FloatSlider(
    value=10, min=0, max=100, step=0.1, description="Exposure Time (ms):"
)
dlam = widgets.FloatSlider(value=3.3, min=0.1, max=500, step=1, description="Bandwidth")

use_qe = widgets.Checkbox(value=True, description="Include Real QE")

altitude = widgets.FloatSlider(
    value=100, min=0.001, max=600, step=0.1, description="Altitude (km)"
)


@widgets.interact(
    apeture=apeture,
    efl=efl,
    pixel_size_x=pixel_size_x,
    pixel_size_y=pixel_size_y,
    int_time=int_time_widget,
    dlam=dlam,
    altitude=altitude,
    use_qe=use_qe,
    detector=widgets.Dropdown(
        options=["CMV2000", "IMX174", "IMX273"], value="IMX273", description="Detector:"
    ),
)
def update(
    apeture,
    efl,
    pixel_size_x,
    pixel_size_y,
    int_time,
    dlam,
    altitude,
    use_qe,
    detector,
):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]

    if use_qe:
        detectorQE = QE[detector](wv)
    else:
        detectorQE = 0.7

    gsd_x = altitude / efl * 1e-3 * pixel_size_x
    gsd_y = altitude / efl * 1e-3 * pixel_size_y

    SN = np.sqrt(
        convertRadAtSatLevelToPhotonsCapturedBySensor(
            radwatts=res,
            waves=wv,
            apeture=apeture * 1e-3,
            efl=efl * 1e-3,
            pixelsize=(pixel_size_x * 1e-6, pixel_size_y * 1e-6),
            int_time=int_time * 1e-3,
            dlam=dlam * 1e-3,
            QE=detectorQE,
            inst_trans=0.8,
        )
    )
    print(SN/detectorQE)
    ax.set_title(
        "F/{:.1f},  Mean S/N: {:.2f}, \n GSD X: {:.1f}cm, GSD Y: {:.1f}cm, Altitude: {:.0f}m".format(
            efl / apeture, np.nanmean(SN), gsd_x * 1000, gsd_y * 1000, altitude
        )
    )
    # ax.set_ylim([0, np.nanmax(SN)*1.1])
    ax.set_xlim([430, 915])
    ax.plot(wv * 1000, SN, wv * 1000, np.sqrt(SN**2/detectorQE))
    #plt.ylim([0, np.max(SN)])
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("S/N")
    plt.legend(["With QE", "S/N"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=4.0, description='Apeture:', max=200.0, min=1.0, step=1.0), FloatSlide…

In [27]:
SN/detectorQE

NameError: name 'SN' is not defined